In [ ]:
# import necessary packages
import numpy as np
import pandas as pd

# path to data we just downloaded
# if you are interested in how modelpoints were created,
# check out the `generating_modelpoints.ipynb` file in the repo
DATA_XLS = "./dstoolkit/model_point.xlsx"

# ingesting our data into a dataframe (df)

### Creating & editting spreadsheets from CLI with `openpyxl` & 'xlwings`

In [4]:
%pip install openpyxl appscript psutil xlwings

Note: you may need to restart the kernel to use updated packages.


In [5]:
import xlwings as xw
import pandas as pd

# Interactive mode: open new workbook
# write pd.DataFrame to "A1" then read it back

book = xw.Book()
sheet = book.sheets[0]
sheet.range("A1").value = pd.DataFrame({"A": [1, 2, 3], "B": [4, 5, 6]})
df = sheet.range("A1").options(pd.DataFrame, expand="table").value


XlwingsError: Make sure to have "appscript" and "psutil", dependencies of xlwings, installed.